# Network eCREST



## Setup

Do the following two setup steps regardless of how you will be using this script. 

### 1. Imports

Run the following code cell to import the necessary packages and modules. 

In [1]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new

from eCREST_cli import ecrest, import_settings

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


In [2]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

dirpath = Path(settings_dict['save_dir'])
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

nodes = [child.name.split('_')[2] for child in sorted(dirpath.iterdir()) 
         if (child.name[0]!='.') & (child.is_file())] # ignore hidden files]

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child
                    


# Create a base_segments dictionary of all cells

base_segments = {}
for x in nodes:
    cell = ecrest(settings_dict,filepath = nodefiles[x],launch_viewer=False)
    base_segments[cell.cell_data['metadata']['main_seg']['base']] = cell.cell_data['base_segments']

In [3]:
viewer = neuroglancer.Viewer()
viewer.set_state({})

'0c1a78469685f66bad2bd84db3affbe1783db805'

In [4]:
location=[17000,17000,1500]

with viewer.config_state.txn() as s:
    s.show_layer_panel = True ###

In [5]:
with viewer.txn(overwrite=True) as s:
           
    dimensions = neuroglancer.CoordinateSpace(
        scales=[16, 16, 30],# self.vx_sizes['em'],
        units='nm',
        names=['x', 'y', 'z']   )

    s.showSlices = False
    s.dimensions = dimensions
    s.position = array(location)
    s.layout = "3d"
    s.projectionScale = 30000
    

In [6]:
with viewer.txn(overwrite=True) as s:
    s.perspectiveViewBackgroundColor= "#ffffff"
    

AttributeError: 'ViewerState' object has no attribute 'perspectiveViewBackgroundColor'

In [7]:
with viewer.txn(overwrite=True) as s:
    wb_open(str(viewer))

In [8]:
db_cursors = sqlite3_connect(settings_dict['db_path'], check_same_thread=False).cursor()

a = ', '.join(['base_address'])

db_cursors.execute(f'''SELECT {a} FROM addresses_table LIMIT 1''')

[base_seg] = db_cursors.fetchall()[0]



In [ ]:
base_seg

In [10]:
two_d_intensity = 0.5

with viewer.txn(overwrite=True) as s:

    s.layers['base_segs'] = neuroglancer.SegmentationLayer(source = base_seg, segments=[], segment_colors={})
    s.layers['base_segs'].ignoreNullVisibleSet = False
    s.layers['base_segs'].pick = False
    s.layers['base_segs'].selectedAlpha = two_d_intensity #For 2D

In [11]:
lf_innet = ['393063300'] #['307591597']
mg1_innet = ['214412684','300210608','387368998','472175645'] # ['389085521','213669553','472409584']
lg_innet = ['215526370']
mg2_innet = ['128473437','215572949','300316308','386117124','472051969','644825148']

In [13]:
cell_color = '#0000ff' # '#0000ff' # '#ff0000' #

for cell_id in mg1_innet:
    this_cell = base_segments[cell_id]
    all_segs = [a for b in this_cell.values() for a in b]

    with viewer.txn(overwrite=True) as s:
        for dtype in this_cell.keys():
            
            color_structure = cell_color # blue
            
            # if dtype=='axon':
            #     color_structure = "#ebecf0"
            
            for bs in this_cell[dtype]:
                s.layers['base_segs'].segments.add(int(bs))
                s.layers['base_segs'].segment_colors[int(bs)] = color_structure # blue

        # s.layers['base_segs'].segment_colors[int(self.cell_data['anchor_seg'])] = '#1e90ff'

In [ ]:
displayed_segs = set([str(x) for x in viewer.state.layers['base_segs'].segments])

In [ ]:
# REMOVE SEGMENTS FROM NGviewer
remove_segs = displayed_segs

with viewer.txn(overwrite=True) as s:
    for bs in remove_segs:
        if int(bs) in s.layers['base_segs'].segments:
            s.layers['base_segs'].segments.remove(int(bs))

In [14]:
download_path = Path('/Users/kperks/Downloads')
filepath = download_path / 'mg1-lf_x-y_.png'

s = viewer.screenshot()
with open(filepath, 'wb') as f:
    f.write(s.screenshot.image)

KeyboardInterrupt: 

In [ ]:
viewer.